In [1]:
import Pkg; Pkg.instantiate()
Pkg.activate(".")
using DrugResponseModel
concs, _, g1s1, g2s1 = load(189, 1);
_, _, g1s2, g2s2 = load(189, 2);
_, _, g1s3, g2s3 = load(189, 3);
g0 = g1s1[1,1,1]+g2s1[1,1,1];
g1m = (g1s1 .+ g1s2 .+ g1s3) ./ 3;
g2m = (g2s1 .+ g2s2 .+ g2s3) ./ 3;


Precompiling project...
  ✓ DrugResponseModel
1 dependency successfully precompiled in 7 seconds (137 already precompiled)
  Activating environment at `~/ODE/DrugResponseModel.jl/Project.toml`


In [2]:
# case 1

function getODEparamspairs(p, conc)
    effects = zeros(eltype(p), 9, length(conc[:, 1]), 2)
    k = 1
    # Scaled drug effect
    for i = 1:2
        xx = 1.0 ./ (1.0 .+ (p[k] ./ conc[:, i]) .^ p[k + 1])
        effects[1, :, i] = p[15] .+ (p[k + 2] - p[15]) .* xx # G1 prog. rate
        effects[2, :, i] = p[16] .+ (p[k + 3] - p[16]) .* xx # G2 prog. rate
        effects[3, :, i] = p[k + 4] .* xx # G1 death rate
        effects[4, :, i] = p[k + 5] .* xx # G2 death rate
        effects[5, :, i] .= p[k + 6] # percentage in G1
        effects[6, :, i] .= p[17]
        effects[7, :, i] .= p[18]
        effects[8, :, i] .= p[19]
        effects[9, :, i] .= p[20]
        
        k += 7
    end
    return effects
end

a, p1 = DrugResponseModel.BBoptim_DrugPairs(concs, g1m, g2m, 4, 1, 2)
pp1 = getODEparamspairs(p1, concs)



Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64}, BlackBoxOptim.RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, BlackBoxOptim.RandomBound{BlackBoxOptim.ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
DE modify state:

Optimization stopped after 100001 steps and 0.27 seconds
Termination reason: Max number of steps (100000) reached
Steps per second = 374760.18
Function evals per second = 374763.93
Improvements/step = 0.99942
Total function evaluations = 100002


Best candidate found: [0.5, 1.0, 1.0e-9, 1.0e-9, 1.0e-9, 1.0e-9, 1.0e-9, 0.5, 1.0, 1.0e-9, 1.0e-9, 1.0e-9, 1.0e-9, 1.0e-9, 2.04577, 1.86946, 31.4292, 5.26844, 49.0575, 10.3015]

Fitness: 0.000000000



9×8×2 Array{Float64, 3}:
[:, :, 1] =
  1.85979       0.185979      0.0974174    …   0.00408337    0.00204372
  1.69951       0.169951      0.089022         0.00373146    0.0018676
  9.09091e-11   9.09091e-10   9.52381e-10      9.98004e-10   9.99001e-10
  9.09091e-11   9.09091e-10   9.52381e-10      9.98004e-10   9.99001e-10
  1.0e-9        1.0e-9        1.0e-9           1.0e-9        1.0e-9
 31.4292       31.4292       31.4292       …  31.4292       31.4292
  5.26844       5.26844       5.26844          5.26844       5.26844
 49.0575       49.0575       49.0575          49.0575       49.0575
 10.3015       10.3015       10.3015          10.3015       10.3015

[:, :, 2] =
  1.85979       0.681922      0.0974174    …   0.00408337    0.00204372
  1.69951       0.623154      0.089022         0.00373146    0.0018676
  9.09091e-11   6.66667e-10   9.52381e-10      9.98004e-10   9.99001e-10
  9.09091e-11   6.66667e-10   9.52381e-10      9.98004e-10   9.99001e-10
  1.0e-9        1.0e-9        1

In [3]:
G1 = zeros(189, 7, 5)
G2 = zeros(189, 7, 5)
t = LinRange(0.0, 95.0, 189)
for k=1:2 # drug number
    for i = 1:7 # concentration number
        G1[:, i, k], G2[:, i, k], _ = predict(pp1[:, i, k], g0, t)
    end
end

p1 = DrugResponseModel.plotavg(G1[:, :, 1], G2[:, :, 1], g1m[:, :, 1], g2m[:, :, 1], 1, :false, concs[1, 1])
p =[DrugResponseModel.plotavg(G1[:, :, 1], G2[:, :, 1], g1m[:, :, 1], g2m[:, :, 1], i, :false, concs[i, 1]) for i=2:7]
plot(p1, p..., figsize=(1000, 700), dpi=150)

LoadError: BoundsError: attempt to access 9×8×2 Array{Float64, 3} at index [1:9, 1, 3]

In [3]:

# case 2
function getODEparamspairs(p, conc)
    effects = zeros(eltype(p), 9, length(conc[:, 1]), 2)
    k = 1
    # Scaled drug effect
    for i = 1:2
        xx = 1.0 ./ (1.0 .+ (p[k] ./ conc[:, i]) .^ p[k + 1])
        effects[1, :, i] = p[k + 2] .+ (p[k + 3] - p[k + 2]) .* xx # G1 prog. rate
        effects[2, :, i] = p[k + 4] .+ (p[k + 5] - p[k + 4]) .* xx # G2 prog. rate
        effects[3, :, i] = p[k + 6] .* xx # G1 death rate
        effects[4, :, i] = p[k + 7] .* xx # G2 death rate
        effects[5, :, i] .= p[k + 8] # percentage in G1
        effects[6, :, i] .= p[19]
        effects[7, :, i] .= p[20]
        effects[8, :, i] .= p[21]
        effects[9, :, i] .= p[22]
        
        k += 9
    end
    return effects
end

a, p2 = DrugResponseModel.BBoptim_DrugPairs(concs, g1m, g2m, 2, 1, 2)
pp2 = getODEparamspairs(p2, conc)


G1 = zeros(189, 7, 5)
G2 = zeros(189, 7, 5)
t = LinRange(0.0, 95.0, 189)
for k=1:5 # drug number
    for i = 1:7 # concentration number
        G1[:, i, k], G2[:, i, k], _ = predict(pp2[:, i, k], g0, t)
    end
end

p1 = DrugResponseModel.plotavg(G1[:, :, 1], G2[:, :, 1], g1m[:, :, 1], g2m[:, :, 1], 1, :false, concs[1, 1])
p =[DrugResponseModel.plotavg(G1[:, :, 1], G2[:, :, 1], g1m[:, :, 1], g2m[:, :, 1], i, :false, concs[i, 1]) for i=2:7]
plot(p1, p..., figsize=(1000, 700), dpi=150)


Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64}, BlackBoxOptim.RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, BlackBoxOptim.RandomBound{BlackBoxOptim.ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
DE modify state:
100.06 secs, 1805 evals, 1710 steps, improv/step: 0.267 (last = 0.2673), fitness=17745.611153867
DE modify state:
200.08 secs, 3467 evals, 3372 steps, improv/step: 0.210 (last = 0.1510), fitness=14921.787698000
DE modify state:
300.09 secs, 5061 evals, 4966 steps, improv/step: 0.189 (last = 0.1455), fitness=12088.602734682
DE modify state:
400.12 secs, 6618 evals, 6523 steps, improv/step: 0.176 (last = 0.1329), fitness=10937.846556139
DE modify state:
500.15 secs, 8210 evals, 8115 steps, improv/step: 0.168 (last = 0.1382), fitness=10642.294962783
DE modify state:
600.17 secs, 9800 evals, 9705 steps, improv/step: 0.164 (last = 0.1440), fitness=10438.043292854
DE modify state:
700.17 secs, 11444 evals,

In [ ]:
# case 3
function getODEparamspairs(p, conc)
    effects = zeros(eltype(p), 9, length(conc[:, 1]), 2)
    k = 1
    # Scaled drug effect
    for i = 1:2
        xx = 1.0 ./ (1.0 .+ (p[k] ./ conc[:, i]) .^ p[k + 1])
        effects[1, :, i] = p[23] .+ (p[k + 2] - p[23]) .* xx # G1 prog. rate
        effects[2, :, i] = p[24] .+ (p[k + 3] - p[24]) .* xx # G2 prog. rate
        effects[3, :, i] = p[k + 4] .* xx # G1 death rate
        effects[4, :, i] = p[k + 5] .* xx # G2 death rate
        effects[5, :, i] .= p[k + 6] # percentage in G1
        effects[6, :, i] .= p[k + 7]
        effects[7, :, i] .= p[k + 8]
        effects[8, :, i] .= p[k + 9]
        effects[9, :, i] .= p[k + 10]
        
        k += 11
    end
    return effects
end

a, p3 = DrugResponseModel.BBoptim_DrugPairs(concs, g1m, g2m, 3, 1, 2)
pp3 = getODEparamspairs(p3, conc)

G1 = zeros(189, 7, 5)
G2 = zeros(189, 7, 5)
t = LinRange(0.0, 95.0, 189)
for k=1:5 # drug number
    for i = 1:7 # concentration number
        G1[:, i, k], G2[:, i, k], _ = predict(pp3[:, i, k], g0, t)
    end
end

p1 = DrugResponseModel.plotavg(G1[:, :, 1], G2[:, :, 1], g1m[:, :, 1], g2m[:, :, 1], 1, :false, concs[1, 1])
p =[DrugResponseModel.plotavg(G1[:, :, 1], G2[:, :, 1], g1m[:, :, 1], g2m[:, :, 1], i, :false, concs[i, 1]) for i=2:7]
plot(p1, p..., figsize=(1000, 700), dpi=150)
